In [ ]:
# Install required libraries
# ! pip install deap plotly pandas scipy pyswarm kaleido

In [18]:
# ! pip install torch torchvision torchaudio

In [ ]:
import random
import time
import numpy as np
from deap import base, creator, tools, algorithms
import plotly.graph_objects as go
import pandas as pd
from scipy.stats import ttest_rel
from scipy.optimize import differential_evolution
from pyswarm import pso
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim

# --- Benchmark Functions ---
def sphere(individual):
    return sum(x**2 for x in individual),

def rastrigin(individual):
    return 10 * len(individual) + sum(x**2 - 10 * np.cos(2 * np.pi * x) for x in individual),

def rosenbrock(individual):
    return sum(100 * (individual[i+1] - individual[i]**2)**2 + (individual[i] - 1)**2 for i in range(len(individual) - 1)),

def ackley(individual):
    n = len(individual)
    sum_sq_term = -0.2 * np.sqrt(sum(x**2 for x in individual) / n)
    cos_term = sum(np.cos(2 * np.pi * x) for x in individual) / n
    return -20 * np.exp(sum_sq_term) - np.exp(cos_term) + 20 + np.e,

def griewank(individual):
    sum_part = sum(x**2 / 4000 for x in individual)
    prod_part = np.prod([np.cos(x / np.sqrt(i + 1)) for i, x in enumerate(individual)])
    return sum_part - prod_part + 1,

def michalewicz(individual):
    m = 10
    total = 0
    for i, x in enumerate(individual):
        total += np.sin(x) * (np.sin((i + 1) * x**2 / np.pi))**(2 * m)
    return -total,

def schwefel(individual):
    n = len(individual)
    return 418.9829 * n - sum(x * np.sin(np.sqrt(np.abs(x))) for x in individual),

def zakharov(individual):
    sum1 = sum(x**2 for x in individual)
    sum2 = sum(0.5 * (i + 1) * x for i, x in enumerate(individual))
    return sum1 + sum2**2 + sum2**4,

# --- Adaptive Penalty Function ---
# Reference: "A Review on Constraint Handling Techniques for Population-based Algorithms" (2022)
# https://link.springer.com/article/10.1007/s11831-022-09859-9
def adaptive_penalty(individual, population, bounds):
    if not population:
        return 0
    total_violations = sum(sum(1 for x in ind if not (bounds[0] <= x <= bounds[1])) for ind in population)
    avg_violation = total_violations / len(population)
    penalty_strength = 1e6 * (1 + avg_violation)
    return sum(penalty_strength for x in individual if not (bounds[0] <= x <= bounds[1]))

# --- Repair Operator ---
# Reference: "A Review on Constraint Handling Techniques for Population-based Algorithms" (2022)
# https://link.springer.com/article/10.1007/s11831-022-09859-9
def repair_individual(individual, bounds):
    return [max(bounds[0], min(bounds[1], x)) for x in individual]

# --- GA Setup ---
def setup_ga(bounds, n_dimensions):
    # Use existing classes if they are already created to avoid errors on re-runs
    if not hasattr(creator, "FitnessMin"):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, *bounds)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_dimensions)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("select", tools.selTournament, tournsize=3)
    return toolbox

# --- Standard GA ---
def run_standard_ga(toolbox, evaluate_func, n_population, n_generations):
    start_time = time.perf_counter()
    toolbox.register("evaluate", evaluate_func)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    population = toolbox.population(n=n_population)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)
    _, logbook = algorithms.eaSimple(
        population, toolbox, cxpb=0.7, mutpb=0.2, ngen=n_generations,
        stats=stats, halloffame=hof, verbose=False
    )
    end_time = time.perf_counter()
    return hof[0].fitness.values[0], logbook, end_time - start_time

# --- Push-Pull Strategy GA (PPS-GA) ---
# Reference: Inspired by push-pull strategy in "Evolutionary constrained multi-objective optimization" (2024)
# https://link.springer.com/article/10.1007/s11831-023-10033-0
class PPSGA:
    def __init__(self, bounds, n_dimensions, population_size, generations):
        self.bounds = bounds
        self.n_dimensions = n_dimensions
        self.population_size = population_size
        self.generations = generations
        self.push_phase = generations // 2
        self.toolbox = setup_ga(bounds, n_dimensions)
        self.toolbox.register("mate", tools.cxBlend, alpha=0.5)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
        self.current_population = []

    def evaluate(self, individual, evaluate_func, gen):
        base_fitness = evaluate_func(individual)[0]
        if gen < self.push_phase:
            return base_fitness,
        penalty = adaptive_penalty(individual, self.current_population, self.bounds)
        return base_fitness + penalty,

    def run(self, evaluate_func):
        start_time = time.perf_counter()
        population = self.toolbox.population(n=self.population_size)
        self.current_population = population

        hof = tools.HallOfFame(1)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)
        logbook = tools.Logbook()
        logbook.header = ["gen", "min", "avg"]

        # Initial evaluation without penalty
        for ind in population:
            ind.fitness.values = (evaluate_func(ind)[0],)
        record = stats.compile(population)
        logbook.record(gen=0, **record)
        hof.update(population)

        for gen in range(1, self.generations + 1):
            self.toolbox.register("evaluate", self.evaluate, evaluate_func=evaluate_func, gen=gen)
            offspring = algorithms.varAnd(population, self.toolbox, cxpb=0.7, mutpb=0.2)
            
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = self.toolbox.map(self.toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit

            population = self.toolbox.select(population + offspring, k=self.population_size)
            self.current_population = population
            
            record = stats.compile(population)
            logbook.record(gen=gen, **record)
            hof.update(population)

        end_time = time.perf_counter()
        final_fitnesses = [self.evaluate(ind, evaluate_func, self.generations)[0] for ind in hof]
        return min(final_fitnesses), logbook, end_time - start_time

# --- RL-Guided Grey Wolf Optimizer (RLCGWO) ---
class RLCGWO:
    """A Reinforcement Learning-Guided Grey Wolf Optimizer (RLCGWO).

    This algorithm enhances the standard Grey Wolf Optimizer (GWO) by using a
    simple, tabular Q-learning agent to dynamically control the exploration and
    exploitation balance. The agent chooses between "exploration" and "exploitation"
    modes, which adjusts a key parameter in the GWO update equations.

    The state for the Q-learning agent is a discretized string representation of the
    population's minimum fitness and diversity.

    Reference:
        The concept is adapted from the paper "RL-GA: A Reinforcement
        Learning-based Genetic Algorithm" (2023), applying the RL control
        mechanism to the GWO framework.
        https://www.frontiersin.org/journals/artificial-intelligence/articles/10.3389/frai.2023.1227843/full

    Args:
        bounds (tuple): A tuple (min_bound, max_bound) for the search space.
        n_dimensions (int): The number of dimensions of the problem.
        population_size (int): The number of wolves in the population.
        generations (int): The number of iterations to run the algorithm.
    """
    def __init__(self, bounds, n_dimensions, population_size, generations):
        self.bounds = bounds
        self.n_dimensions = n_dimensions
        self.population_size = population_size
        self.generations = generations
        self.a = 2.0
        self.actions = ["exploration", "exploitation"]
        self.epsilon, self.alpha, self.gamma = 0.2, 0.1, 0.9
        self.current_action = "balanced"
        self.q_table = defaultdict(lambda: 0)
        self.current_population = []

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.choice(self.actions)
        q_values = [self.q_table[(state, action)] for action in self.actions]
        return self.actions[np.argmax(q_values)]

    def update_q_table(self, state, action, reward, next_state):
        max_next_q = max(self.q_table.get((next_state, a), 0) for a in self.actions)
        self.q_table[(state, action)] += self.alpha * (reward + self.gamma * max_next_q - self.q_table[(state, action)])

    def evaluate(self, individual, evaluate_func):
        base_fitness = evaluate_func(individual)[0]
        penalty = adaptive_penalty(individual, self.current_population, self.bounds)
        return base_fitness + penalty

    def run(self, evaluate_func):
        start_time = time.perf_counter()
        population = [repair_individual([random.uniform(self.bounds[0], self.bounds[1]) for _ in range(self.n_dimensions)], self.bounds) for _ in range(self.population_size)]
        self.current_population = population

        stats = tools.Statistics(key=lambda ind: self.evaluate(ind, evaluate_func))
        stats.register("min", np.min)
        stats.register("avg", np.mean)
        stats.register("diversity", np.std)
        logbook = tools.Logbook()
        logbook.header = ["gen", "min", "avg", "diversity"]
        
        record = stats.compile(population)
        logbook.record(gen=0, **record)
        state = f"{record['min']:.2f}_{record['diversity']:.2f}"

        for gen in range(1, self.generations + 1):
            self.a = 2 * (1 - gen / self.generations)
            self.current_action = self.choose_action(state)
            a_factor = 1.5 if self.current_action == "exploration" else 0.5
            
            sorted_pop = sorted(population, key=lambda ind: self.evaluate(ind, evaluate_func))
            alpha, beta, delta = sorted_pop[0], sorted_pop[1], sorted_pop[2]
            
            new_population = []
            for i in range(self.population_size):
                A1 = a_factor * (2 * np.random.random(self.n_dimensions) - 1)
                A2 = a_factor * (2 * np.random.random(self.n_dimensions) - 1)
                A3 = a_factor * (2 * np.random.random(self.n_dimensions) - 1)
                C1, C2, C3 = 2 * np.random.random(self.n_dimensions), 2 * np.random.random(self.n_dimensions), 2 * np.random.random(self.n_dimensions)

                D_alpha = np.abs(C1 * np.array(alpha) - np.array(population[i]))
                D_beta = np.abs(C2 * np.array(beta) - np.array(population[i]))
                D_delta = np.abs(C3 * np.array(delta) - np.array(population[i]))
                
                X1 = np.array(alpha) - A1 * D_alpha
                X2 = np.array(beta) - A2 * D_beta
                X3 = np.array(delta) - A3 * D_delta
                
                new_individual = (X1 + X2 + X3) / 3.0
                new_population.append(repair_individual(new_individual.tolist(), self.bounds))
            
            population = new_population
            self.current_population = population
            
            record = stats.compile(population)
            logbook.record(gen=gen, **record)
            reward = -record["min"] + 0.1 * record["diversity"]
            next_state = f"{record['min']:.2f}_{record['diversity']:.2f}"
            self.update_q_table(state, self.current_action, reward, next_state)
            state = next_state

        end_time = time.perf_counter()
        final_fitnesses = [self.evaluate(ind, evaluate_func) for ind in population]
        return min(final_fitnesses), logbook, end_time - start_time

# --- Opposition-Based Learning Function ---
def opposition_based_learning(population, bounds):
    opposite_population = []
    for individual in population:
        opposite_individual = []
        for i, dim in enumerate(individual):
            opposite_dim = (bounds[0] + bounds[1]) - dim
            opposite_individual.append(opposite_dim)
        opposite_population.append(repair_individual(opposite_individual, bounds))
    return opposite_population

# --- Dueling Deep Q-Network Architecture ---
class DuelingDQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DuelingDQN, self).__init__()
        self.feature_layer = nn.Sequential(
            nn.Linear(state_size, 64),
            nn.ReLU()
        )
        self.value_stream = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.advantage_stream = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_size)
        )

    def forward(self, state):
        features = self.feature_layer(state)
        value = self.value_stream(features)
        advantages = self.advantage_stream(features)
        q_values = value + (advantages - advantages.mean(dim=1, keepdim=True))
        return q_values

# --- Enhanced RL-Guided Grey Wolf Optimizer (RLCGWO) ---

class EnhancedRLGWO:
    """An Enhanced Reinforcement Learning-Guided Grey Wolf Optimizer (EnhancedRLGWO).

This class implements a state-of-the-art hybrid metaheuristic that combines the
Grey Wolf Optimizer (GWO) with a sophisticated Deep Q-Network (DQN) agent.
The agent learns a policy to dynamically control the GWO's search behavior,
intelligently deciding when to exploit known solutions versus when to explore
new regions of the search space.

Key Enhancements:
- **Dueling DQN Architecture**: Utilizes a neural network with separate streams
  for state-value and action-advantage functions for more robust learning.
- **Prioritized Experience Replay (PER)**: Employs a memory buffer that
  prioritizes replaying more informative experiences, accelerating learning.
- **Target Network**: Uses a second, slow-updating network to provide stable
  targets for the Q-learning updates.
- **Strategic Operator Selection**: The DQN agent's primary role is to choose
  between standard GWO update modes and a powerful strategic escape maneuver.
- **Opposition-Based Learning (OBL)**: The agent can trigger an OBL phase,
  creating a mirror-image population to escape local optima and perform
  a large-scale jump in the search space.
- **Adaptive Exploration**: Uses an epsilon-decay schedule to transition
  smoothly from an initial exploratory phase to a final exploitative phase.

Args:
    bounds (tuple): A tuple (min_bound, max_bound) for the search space.
    n_dimensions (int): The number of dimensions of the problem.
    population_size (int): The number of wolves in the population.
    generations (int): The number of iterations to run the algorithm.
"""
    def __init__(self, bounds, n_dimensions, population_size, generations):
        self.bounds = bounds
        self.n_dimensions = n_dimensions
        self.population_size = population_size
        self.generations = generations
        
        self.actions = [
            ("set_a", 0.5),   # Strong exploitation
            ("set_a", 1.0),   # Balanced
            ("set_a", 1.5),   # Strong exploration
            ("activate_obl", None) # Strategic jump
        ]
        
        self.epsilon_start, self.epsilon_end, self.epsilon_decay = 1.0, 0.01, 0.995
        self.epsilon = self.epsilon_start
        self.gamma = 0.99
        self.initial_optimizer_lr = 0.001
        self.lr_decay_rate = 0.01
        self.replay_buffer_size = 2000
        self.batch_size = 64
        self.beta = 0.4
        self.priority_epsilon = 1e-6
        self.tau = 0.005
        self.replay_buffer = []
        self.current_population = []
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        state_size = 3
        action_size = len(self.actions)
        
        self.q_network = DuelingDQN(state_size, action_size).to(self.device)
        self.target_q_network = DuelingDQN(state_size, action_size).to(self.device)
        self.target_q_network.load_state_dict(self.q_network.state_dict())
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=self.initial_optimizer_lr)

    def add_to_buffer(self, experience):
        max_priority = max([p for p, _ in self.replay_buffer]) if self.replay_buffer else 1.0
        self.replay_buffer.append((max_priority, experience))
        if len(self.replay_buffer) > self.replay_buffer_size:
            self.replay_buffer.pop(0)

    def replay(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        priorities = np.array([p for p, _ in self.replay_buffer])
        probs = priorities ** self.beta
        probs /= probs.sum()

        indices = np.random.choice(len(self.replay_buffer), self.batch_size, p=probs)
        batch = [self.replay_buffer[i][1] for i in indices]

        total = len(self.replay_buffer)
        weights = (total * probs[indices]) ** (-self.beta)
        weights /= weights.max()
        weights = torch.tensor(weights, dtype=torch.float32).to(self.device)

        states, actions, rewards, next_states = zip(*batch)

        states = torch.tensor(np.array(states), dtype=torch.float32).to(self.device)
        next_states = torch.tensor(np.array(next_states), dtype=torch.float32).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
        action_indices = torch.tensor(actions, dtype=torch.int64).to(self.device)

        q_values = self.q_network(states)
        predicted_q = q_values.gather(1, action_indices.unsqueeze(1)).squeeze(1)

        with torch.no_grad():
            next_q_values = self.target_q_network(next_states).max(1)[0]
        
        target_q = rewards + self.gamma * next_q_values
        td_errors = torch.abs(target_q - predicted_q).detach()
        loss = (weights * nn.MSELoss(reduction='none')(predicted_q, target_q)).mean()
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update_target_network()

        for i, idx in enumerate(indices):
            priority = td_errors[i].item() + self.priority_epsilon
            self.replay_buffer[idx] = (priority, self.replay_buffer[idx][1])

    def soft_update_target_network(self):
        for target_param, local_param in zip(self.target_q_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(self.tau*local_param.data + (1.0-self.tau)*target_param.data)

    def choose_action_index(self, state):
        if random.random() < self.epsilon:
            return random.randrange(len(self.actions))
        state_tensor = torch.tensor(state, dtype=torch.float32).to(self.device).unsqueeze(0)
        with torch.no_grad():
            q_values = self.q_network(state_tensor)
        return torch.argmax(q_values).item()

    def evaluate(self, individual, evaluate_func):
        base_fitness = evaluate_func(individual)[0]
        penalty = adaptive_penalty(individual, self.current_population, self.bounds)
        return base_fitness + penalty

    def run(self, evaluate_func):
        start_time = time.perf_counter()
        population = [repair_individual([random.uniform(self.bounds[0], self.bounds[1]) for _ in range(self.n_dimensions)], self.bounds) for _ in range(self.population_size)]
        self.current_population = population

        logbook = tools.Logbook()
        logbook.header = ["gen", "min", "avg", "diversity"]
        
        fitnesses = [self.evaluate(ind, evaluate_func) for ind in population]
        min_fit, avg_fit, std_fit = np.min(fitnesses), np.mean(fitnesses), np.std(fitnesses)
        logbook.record(gen=0, min=min_fit, avg=avg_fit, diversity=std_fit)
        state = [min_fit, avg_fit, std_fit]

        for gen in range(1, self.generations + 1):
            new_optimizer_lr = self.initial_optimizer_lr * np.exp(-self.lr_decay_rate * gen)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = new_optimizer_lr
            self.beta = min(1.0, self.beta + 0.001)

            action_index = self.choose_action_index(state)
            action_type, action_value = self.actions[action_index]

            if action_type == "activate_obl":
                opposite_pop = opposition_based_learning(population, self.bounds)
                combined_pop = population + opposite_pop
                
                combined_fitnesses = [self.evaluate(ind, evaluate_func) for ind in combined_pop]
                sorted_combined = sorted(zip(combined_pop, combined_fitnesses), key=lambda x: x[1])
                
                population = [ind for ind, fit in sorted_combined[:self.population_size]]
                fitnesses = [fit for ind, fit in sorted_combined[:self.population_size]]
            else: # action_type == "set_a"
                a_factor = action_value
                pop_with_fitness = list(zip(population, fitnesses))
                sorted_pop_with_fitness = sorted(pop_with_fitness, key=lambda x: x[1])
                sorted_pop = [ind for ind, fit in sorted_pop_with_fitness]
                alpha_wolf, beta_wolf, delta_wolf = sorted_pop[0], sorted_pop[1], sorted_pop[2]
                
                new_population = []
                for i in range(self.population_size):
                    A1,A2,A3 = a_factor*(2*np.random.random(self.n_dimensions)-1), a_factor*(2*np.random.random(self.n_dimensions)-1), a_factor*(2*np.random.random(self.n_dimensions)-1)
                    C1,C2,C3 = 2*np.random.random(self.n_dimensions), 2*np.random.random(self.n_dimensions), 2*np.random.random(self.n_dimensions)

                    D_alpha = np.abs(C1*np.array(alpha_wolf) - np.array(population[i]))
                    D_beta = np.abs(C2*np.array(beta_wolf) - np.array(population[i]))
                    D_delta = np.abs(C3*np.array(delta_wolf) - np.array(population[i]))
                    
                    X1,X2,X3 = np.array(alpha_wolf)-A1*D_alpha, np.array(beta_wolf)-A2*D_beta, np.array(delta_wolf)-A3*D_delta
                    
                    new_individual = (X1 + X2 + X3) / 3.0
                    new_population.append(repair_individual(new_individual.tolist(), self.bounds))
                population = new_population

            self.current_population = population
            fitnesses = [self.evaluate(ind, evaluate_func) for ind in population]
            min_fit, avg_fit, std_fit = np.min(fitnesses), np.mean(fitnesses), np.std(fitnesses)
            logbook.record(gen=gen, min=min_fit, avg=avg_fit, diversity=std_fit)
            
            reward = -min_fit + 0.1 * std_fit
            next_state = [min_fit, avg_fit, std_fit]

            self.add_to_buffer((state, action_index, reward, next_state))
            self.replay()
            state = next_state
            
            self.epsilon = max(self.epsilon_end, self.epsilon_decay * self.epsilon)

        end_time = time.perf_counter()
        return min(fitnesses), logbook, end_time - start_time

# --- Other Optimization Algorithms ---
def run_de(evaluate_func, bounds, n_dimensions, n_generations):
    start_time = time.perf_counter()
    bounds_list = [(bounds[0], bounds[1])] * n_dimensions
    result = differential_evolution(
        lambda x: evaluate_func(x)[0], bounds_list, maxiter=n_generations, popsize=15, polish=False
    )
    end_time = time.perf_counter()
    return result.fun, None, end_time - start_time

def run_pso(evaluate_func, bounds, n_dimensions, n_generations):
    start_time = time.perf_counter()
    lb = [bounds[0]] * n_dimensions
    ub = [bounds[1]] * n_dimensions
    xopt, fopt = pso(lambda x: evaluate_func(x)[0], lb, ub, maxiter=n_generations*2, swarmsize=50) # PSO often needs more evaluations
    end_time = time.perf_counter()
    return fopt, None, end_time - start_time

# --- Statistical Analysis and Visualization ---
def create_plots_and_stats(results, functions, algorithms, complexity_data):
    summary_data = []
    for func_name in functions:
        row = {"Function": func_name}
        for algo_name in algorithms:
            fitnesses = results[algo_name][func_name]["fitness"]
            times = results[algo_name][func_name]["time"]
            row.update({
                f"{algo_name} Mean Fitness": np.mean(fitnesses),
                f"{algo_name} Std Fitness": np.std(fitnesses),
                f"{algo_name} Mean Time (s)": np.mean(times)
            })
        
        # Perform t-tests against a baseline algorithm if it exists
        base_algo = "GA" 
        if base_algo in results and base_algo in results[list(results.keys())[0]]:
            base_fitnesses = results[base_algo][func_name]["fitness"]
            for algo_name in algorithms:
                if algo_name != base_algo and algo_name in results and func_name in results[algo_name]:
                    other_fitnesses = results[algo_name][func_name]["fitness"]
                    try:
                        t_stat, p_value = ttest_rel(base_fitnesses, other_fitnesses)
                        significant = p_value < 0.05
                    except ValueError:
                        p_value = 1.0
                        significant = False
                    row.update({
                        f"P-Value ({base_algo} vs {algo_name})": p_value,
                        f"Significant ({base_algo} vs {algo_name})": significant
                    })
        
        summary_data.append(row)

        # Plot convergence for each function
        fig = go.Figure()
        for algo_name in algorithms:
            if algo_name not in ["DE", "PSO"]:
                if algo_name in results and func_name in results[algo_name]:
                    logbooks = results[algo_name][func_name]["logbook"]
                    if logbooks:
                        min_fitness_runs = []
                        max_len = 0
                        for logbook in logbooks:
                            if logbook:
                                min_fitness_runs.append(logbook.select("min"))
                                max_len = max(max_len, len(logbook.select("gen")))
                        
                        if min_fitness_runs:
                            avg_fitness = np.full(max_len, np.nan)
                            for i in range(max_len):
                                vals = [run[i] for run in min_fitness_runs if i < len(run)]
                                if vals:
                                    avg_fitness[i] = np.mean(vals)

                            gens = list(range(max_len))
                            fig.add_trace(go.Scatter(x=gens, y=avg_fitness, mode="lines", name=algo_name))

        fig.update_layout(
            title=f"<b>Average Convergence Plot for {func_name}</b>",
            xaxis_title="Generation",
            yaxis_title="Average Best Fitness (Lower is Better)",
            template="plotly_white",
            legend_title="Algorithm"
        )
        fig.show()


    # Create and print summary DataFrame
    results_df = pd.DataFrame(summary_data)
    print("\n--- Results DataFrame ---")
    pd.set_option('display.float_format', lambda x: '%.3e' % x)
    print(results_df)

    # Save summary to CSV
    results_df.to_csv("optimization_results.csv", index=False)

    # Fitness Bar Chart
    fig_fitness = go.Figure()
    for algo_name in algorithms:
        fig_fitness.add_trace(go.Bar(
            name=algo_name,
            x=results_df["Function"],
            y=results_df[f"{algo_name} Mean Fitness"],
            error_y=dict(type="data", array=results_df[f"{algo_name} Std Fitness"])
        ))
    fig_fitness.update_layout(
        barmode="group",
        title_text="<b>Mean Fitness Comparison Across Algorithms</b>",
        xaxis_title="Benchmark Function",
        yaxis_title="Mean Best Fitness (Log Scale)",
        yaxis_type="log",
        legend_title="Algorithm",
        template="plotly_white"
    )
    fig_fitness.show()

    # Time Bar Chart
    fig_time = go.Figure()
    for algo_name in algorithms:
        fig_time.add_trace(go.Bar(
            name=algo_name,
            x=results_df["Function"],
            y=results_df[f"{algo_name} Mean Time (s)"]
        ))
    fig_time.update_layout(
        barmode="group",
        title_text="<b>Mean Execution Time Comparison</b>",
        xaxis_title="Benchmark Function",
        yaxis_title="Mean Execution Time (seconds)",
        legend_title="Algorithm",
        template="plotly_white"
    )
    fig_time.show()

# --- Main Execution Block ---
def main():
    # --- Experiment Setup ---
    N_DIMENSIONS = 10
    N_POPULATION = 50
    N_GENERATIONS = 100
    N_RUNS = 5

    FUNCTIONS = {
        "Sphere": (sphere, (-5.12, 5.12)),
        "Rastrigin": (rastrigin, (-5.12, 5.12)),
        "Rosenbrock": (rosenbrock, (-5, 10)),
        "Ackley": (ackley, (-32.768, 32.768)),
        "Griewank": (griewank, (-600, 600)),
        "Michalewicz": (michalewicz, (0, np.pi)),
    }

    ALGORITHMS = {
        "GA": lambda toolbox, func: run_standard_ga(toolbox, FUNCTIONS[func][0], N_POPULATION, N_GENERATIONS),
        "PPS-GA": lambda toolbox, func: PPSGA(FUNCTIONS[func][1], N_DIMENSIONS, N_POPULATION, N_GENERATIONS).run(FUNCTIONS[func][0]),
        "RLCGWO": lambda toolbox, func: RLCGWO(FUNCTIONS[func][1], N_DIMENSIONS, N_POPULATION, N_GENERATIONS).run(FUNCTIONS[func][0]),
        "EnhancedRLGWO": lambda toolbox, func: EnhancedRLGWO(FUNCTIONS[func][1], N_DIMENSIONS, N_POPULATION, N_GENERATIONS).run(FUNCTIONS[func][0]),
        "DE": lambda toolbox, func: run_de(FUNCTIONS[func][0], FUNCTIONS[func][1], N_DIMENSIONS, N_GENERATIONS // 15),
        "PSO": lambda toolbox, func: run_pso(FUNCTIONS[func][0], FUNCTIONS[func][1], N_DIMENSIONS, N_GENERATIONS // 2),
    }

    COMPLEXITY = {
        "GA": "O(G*P*D)",
        "PPS-GA": "O(G*P*(D+P))",
        "RLCGWO": "O(G*P*(D+P))",
        "EnhancedRLGWO": "O(G*P*(D+P) + G*RL)",
        "DE": "O(G*P*D)",
        "PSO": "O(G*P*D)",
    }

    # --- Run Experiments ---
    results = {algo: {func: {"fitness": [], "time": [], "logbook": []} for func in FUNCTIONS} for algo in ALGORITHMS}
    toolbox = setup_ga((-10, 10), N_DIMENSIONS)  # Default bounds for GA setup

    for func_name in FUNCTIONS:
        print(f"🧪 Running experiments for {func_name}...")
        for algo_name in ALGORITHMS:
            print(f"  -> Algorithm: {algo_name}")
            for i in range(N_RUNS):
                fitness, logbook, time_taken = ALGORITHMS[algo_name](toolbox, func_name)
                results[algo_name][func_name]["fitness"].append(fitness)
                results[algo_name][func_name]["time"].append(time_taken)
                results[algo_name][func_name]["logbook"].append(logbook)

    # --- Analysis and Visualization ---
    create_plots_and_stats(results, FUNCTIONS.keys(), ALGORITHMS.keys(), COMPLEXITY)

    print("\nComputational Complexity (G=Generations, P=Population, D=Dimensions, RL=RL update cost):")
    for algo, complexity in COMPLEXITY.items():
        print(f"- {algo}: {complexity}")

if __name__ == "__main__":
    main()

🧪 Running experiments for Sphere...
  -> Algorithm: GA
  -> Algorithm: PPS-GA
  -> Algorithm: RLCGWO
  -> Algorithm: EnhancedRLGWO
  -> Algorithm: DE
  -> Algorithm: PSO
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: Swarm best objective change less than 1e-08
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
🧪 Running experiments for Rastrigin...
  -> Algorithm: GA
  -> Algorithm: PPS-GA
  -> Algorithm: RLCGWO
  -> Algorithm: EnhancedRLGWO
  -> Algorithm: DE
  -> Algorithm: PSO
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: Swarm best objective change less than 1e-08
🧪 Running experiments for Rosenbrock...
  -> Algorithm: GA
  -> Algorithm: PPS-GA
  -> Algorithm: RLC


--- Results DataFrame ---
      Function  GA Mean Fitness  GA Std Fitness  GA Mean Time (s)  \
0       Sphere        2.321e-03       2.510e-03         4.003e-02   
1    Rastrigin        9.200e+00       2.814e+00         5.594e-02   
2   Rosenbrock        5.120e+01       2.478e+01         4.417e-02   
3       Ackley        2.131e-02       1.320e-02         7.476e-02   
4     Griewank        3.650e-02       3.512e-02         7.989e-02   
5  Michalewicz       -8.454e+00       5.524e-01         6.613e-02   

   PPS-GA Mean Fitness  PPS-GA Std Fitness  PPS-GA Mean Time (s)  \
0            1.491e-02           1.321e-02             6.966e-02   
1            1.043e+01           2.967e+00             1.014e-01   
2            8.359e+01           9.978e+01             1.032e-01   
3            7.828e-02           4.642e-02             8.914e-02   
4            1.965e+00           2.710e+00             1.466e-01   
5           -8.360e+00           4.226e-01             1.179e-01   

   RLCGWO Me


Computational Complexity (G=Generations, P=Population, D=Dimensions, RL=RL update cost):
- GA: O(G*P*D)
- PPS-GA: O(G*P*(D+P))
- RLCGWO: O(G*P*(D+P))
- EnhancedRLGWO: O(G*P*(D+P) + G*RL)
- DE: O(G*P*D)
- PSO: O(G*P*D)
